# Securing the Kubernetes API server

## Understanding authentication

In the previous chapter, we said the API server can be configured with one or more authentication plugins (and the same is true for authorization plugins). When a request is received by the API server, it goes through the list of authentication plugins, so they can each examine the request and try to determine who’s sending the request. The first plugin that can extract that information from the request returns the username, user ID, and the groups the client belongs to back to the API server core. The API server stops invoking the remaining authentication plugins and continues onto the authorization phase.

Several authentication plugins are available. They obtain the identity of the client using the following methods:
- From the client certificate
- From an authentication token passed in an HTTP header
- Basic HTTP authentication
- Others

The authentication plugins are enabled through command-line options when starting the API server.

### Users and groups

An authentication plugin returns the username and group(s) of the authenticated user. Kubernetes doesn’t store that information anywhere; it uses it to verify whether the user is authorized to perform an action or not.

**Understanding users**

Kubernetes distinguishes between two kinds of clients connecting to the API server:
- Actual humans (users)
- Pods (more specifically, applications running inside them)

Both these types of clients are authenticated using the aforementioned authentication plugins. Users are meant to be managed by an external system, such as a Single Sign On (SSO) system, but the pods use a mechanism called service accounts, which are created and stored in the cluster as ServiceAccount resources. In contrast, no resource represents user accounts, which means you can’t create, update, or delete users through the API server.

We won’t go into any details of how to manage users, but we will explore Service-Accounts in detail, because they’re essential for running pods. For more information on how to configure the cluster for authentication of human users, cluster administrators should refer to the Kubernetes Cluster Administrator guide at http://kubernetes.io/docs/admin.

**Understanding groups**

Both human users and ServiceAccounts can belong to one or more groups. We’ve said that the authentication plugin returns groups along with the username and user ID. Groups are used to grant permissions to several users at once, instead of having to grant them to individual users.

Groups returned by the plugin are nothing but strings, representing arbitrary group names, but built-in groups have special meaning:
- The `system:unauthenticated` group is used for requests where none of the authentication plugins could authenticate the client.
- The `system:authenticated` group is automatically assigned to a user who was authenticated successfully.
- The `system:serviceaccounts` group encompasses all ServiceAccounts in the system.
- The `system:serviceaccounts:<namespace>` includes all ServiceAccounts in a specific namespace.

###  Introducing ServiceAccounts

Let’s explore ServiceAccounts up close. You’ve already learned that the API server requires clients to authenticate themselves before they’re allowed to perform operations on the server. And you’ve already seen how pods can authenticate by sending the contents of the `file/var/run/secrets/kubernetes.io/serviceaccount/token`, which is mounted into each container’s filesystem through a secret volume.

But what exactly does that file represent? Every pod is associated with a Service-Account, which represents the identity of the app running in the pod. The token file holds the ServiceAccount’s authentication token. When an app uses this token to connect to the API server, the authentication plugin authenticates the ServiceAccount and passes the ServiceAccount’s username back to the API server core. Service-Account usernames are formatted like this:

    system:serviceaccount:<namespace>:<service account name>

The API server passes this username to the configured authorization plugins, which determine whether the action the app is trying to perform is allowed to be performed by the ServiceAccount.

ServiceAccounts are nothing more than a way for an application running inside a pod to authenticate itself with the API server. As already mentioned, applications do that by passing the ServiceAccount’s token in the request.

**Understanding the ServiceAccount resource**

ServiceAccounts are resources just like Pods, Secrets, ConfigMaps, and so on, and are scoped to individual namespaces. A default ServiceAccount is automatically created for each namespace (that’s the one your pods have used all along).

You can list ServiceAccounts like you do other resources:

    kubectl get sa

As you can see, the current namespace only contains the default ServiceAccount. Additional ServiceAccounts can be added when required. Each pod is associated with exactly one ServiceAccount, but multiple pods can use the same ServiceAccount. As you can see in figure 12.1, a pod can only use a ServiceAccount from the same namespace.

<img alt="" src="https://dpzbhybb2pdcj.cloudfront.net/luksa/Figures/12fig01_alt.jpg" data-action="zoom" data-zoom-src="https://dpzbhybb2pdcj.cloudfront.net/luksa/HighResolutionFigures/figure_12-1.png" class="medium-zoom-image">

**Understanding how ServiceAccounts tie into authorization**

You can assign a ServiceAccount to a pod by specifying the account’s name in the pod manifest. If you don’t assign it explicitly, the pod will use the default ServiceAccount in the namespace.

By assigning different ServiceAccounts to pods, you can control which resources each pod has access to. When a request bearing the authentication token is received by the API server, the server uses the token to authenticate the client sending the request and then determines whether or not the related ServiceAccount is allowed to perform the requested operation. The API server obtains this information from the system-wide authorization plugin configured by the cluster administrator. One of the available authorization plugins is the role-based access control (RBAC) plugin, which is discussed later in this chapter. From Kubernetes version 1.6 on, the RBAC plugin is the plugin most clusters should use.

###  Creating ServiceAccounts

We’ve said every namespace contains its own default ServiceAccount, but additional ones can be created if necessary. But why should you bother with creating Service-Accounts instead of using the default one for all your pods?

The obvious reason is cluster security. Pods that don’t need to read any cluster metadata should run under a constrained account that doesn’t allow them to retrieve or modify any resources deployed in the cluster. Pods that need to retrieve resource metadata should run under a ServiceAccount that only allows reading those objects’ metadata, whereas pods that need to modify those objects should run under their own ServiceAccount allowing modifications of API objects.

Let’s see how you can create additional ServiceAccounts, how they relate to Secrets, and how you can assign them to your pods.

**Creating a ServiceAccount**

Creating a ServiceAccount is incredibly easy, thanks to the dedicated kubectl create serviceaccount command. Let’s create a new ServiceAccount called foo:

    kubectl create serviceaccount foo

Now, you can inspect the ServiceAccount with the describe command, as shown in the following listing.

    kubectl describe sa foo

You can see that a custom token Secret has been created and associated with the ServiceAccount. If you look at the Secret’s data with kubectl describe secret foo-token-qzq7j, you’ll see it contains the same items (the CA certificate, namespace, and token) as the default ServiceAccount’s token does (the token itself will obviously be different), as shown in the following listing.

    kubectl describe secret foo-token-96pl4

> NOTE: You’ve probably heard of JSON Web Tokens (JWT). The authentication tokens used in ServiceAccounts are JWT tokens.

**Understanding a ServiceAccount’s mountable secrets**

The token is shown in the Mountable secrets list when you inspect a ServiceAccount with kubectl describe. Let me explain what that list represents. In chapter 7 you learned how to create Secrets and mount them inside a pod. By default, a pod can mount any Secret it wants. But the pod’s ServiceAccount can be configured to only allow the pod to mount Secrets that are listed as mountable Secrets on the Service-Account. To enable this feature, the ServiceAccount must contain the following annotation: `kubernetes.io/enforce-mountable-secrets="true"`.

If the ServiceAccount is annotated with this annotation, any pods using it can mount only the ServiceAccount’s mountable Secrets—they can’t use any other Secret.

**Understanding a ServiceAccount’s image pull Secrets**

A ServiceAccount can also contain a list of image pull Secrets, which we examined in chapter 7. In case you don’t remember, they are Secrets that hold the credentials for pulling container images from a private image repository.

The following listing shows an example of a ServiceAccount definition, which includes the image pull Secret you created in chapter 7.

    apiVersion: v1
    kind: ServiceAccount
    metadata:
      name: my-service-account
    imagePullSecrets:
    - name: my-dockerhub-secret

A ServiceAccount’s image pull Secrets behave slightly differently than its mountable Secrets. Unlike mountable Secrets, they don’t determine which image pull Secrets a pod can use, but which ones are added automatically to all pods using the Service-Account. Adding image pull Secrets to a ServiceAccount saves you from having to add them to each pod individually.

### Assigning a ServiceAccount to a pod

After you create additional ServiceAccounts, you need to assign them to pods. This is done by setting the name of the ServiceAccount in the spec.serviceAccountName field in the pod definition.

> NOTE: A pod’s ServiceAccount must be set when creating the pod. It can’t be changed later.

**Creating a pod which uses a custom ServiceAccount**

In chapter 8 you deployed a pod that ran a container based on the tutum/curl image and an ambassador container alongside it. You used it to explore the API server’s REST interface. The ambassador container ran the kubectl proxy process, which used the pod’s ServiceAccount’s token to authenticate with the API server.

You can now modify the pod so it uses the foo ServiceAccount you created minutes ago. The next listing shows the pod definition.